In [1]:
import pandas as pd
import json
import re
import os
import time
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from pythainlp.tokenize import word_tokenize, sent_tokenize
from pythainlp.util import normalize
from pythainlp.util import Trie
from pythainlp.corpus.common import thai_stopwords
from pythainlp.corpus.common import thai_words

from datetime import datetime
import googleapiclient.discovery

In [4]:
df = pd.read_excel('Corpus_raw_data.xlsx')

In [5]:
df

,textDisplay,sentiment
0,ชอบกล้อง กับ ตัดต่อวีดีโอแค่นั้นแหละถ้าจะซื้อไ...,pos
1,อยากทราบวิธีรักษาแบต 14 กับความร้อนของเครื่องค...,neg
2,นี่ซื้อมาใช้ได้ 1 เดือน คุณภาพแบตลดไป 3% เหลือ...,NaN
3,กล้องวูบวาบเป็นเพราะอะไรคะ,neg
4,ขอถามหน่อยพี่ ผมใช้ 14 พอปิดหน้าจอ มันจะขึ้นวง...,NaN
...,...,...
1285,ข้อมูลครบถ้วน มีลูกเล่นในการนำเสนอภาพได้สวยงาม...,pos
1286,ใครคลิปเสื้อสีขาวนี่เเฟนถ่ายให้ปล่าวครัย555,NaN
1287,ถ้าถ่ายทั่วไป max จะดีกว่านิดหน่อย แต่ถ่ายตอนก...,NaN
1288,ภาพปกคลิป น้อนออย น่ารักกกกก🥰🥰🥰,pos


In [6]:
with open('model_smartphone.json', 'r', encoding="utf-8") as file:
    model_ip = json.load(file)

In [22]:
with open('Question_lexicon.json', 'r', encoding="utf-8") as file:
    question_list = json.load(file)

In [8]:
question_cm = {i for i in question_list}

In [9]:
model_word = []
for i in model_ip:
    model_word.append(i)
    for j in model_ip[i]:
        model_word.append(j)

model = {i for i in model_word}

In [10]:
keywords_option = ["หน้าจอ","จอ", "แบต", "แบตเตอรี่", "กล้อง", "ถ่าย","ถ่ายภาพ", "วีดีโอ", "รูปถ่าย","hz","ถ่ายคลิป","dynamic island","fps","กล้องหน้า","กล้องหลัง","focus"]
keywords = keywords_option
print(keywords)

def keyword_filter(data):
    
    relevant_words = [word for word in data if word in keywords]
    if not relevant_words:
        return None
    
    data = "".join(word for word in data)
    return data

['หน้าจอ', 'จอ', 'แบต', 'แบตเตอรี่', 'กล้อง', 'ถ่าย', 'ถ่ายภาพ', 'วีดีโอ', 'รูปถ่าย', 'hz', 'ถ่ายคลิป', 'dynamic island', 'fps', 'กล้องหน้า', 'กล้องหลัง', 'focus']


In [23]:
def question_filter(dataframe, text_column):
    # Create a boolean mask to identify rows with question words
    mask = dataframe[text_column].apply(lambda text: any(word in text for word in question_list))
    # Invert the mask to keep only rows without question words
    dataframe = dataframe.loc[~mask]

    return dataframe

In [12]:
custom_word = model.union(question_list)
# print(custom_word)
words = custom_word.union(thai_words())
custom_dictionary_trie = Trie(words)

def word_token(data):
    data = "".join(u.lower() for u in data if u not in ("?", ".", ";", ":", "!", '"', "ๆ", "ฯ",
                    "/", "<", ">", "^", "-", "*", "(", ")", "%", "_", ",", "#", "'", "{", "}", "…","&"," "))
    data = normalize(data)
    data = word_tokenize(data, custom_dict=custom_dictionary_trie, keep_whitespace=False)

    return data

# ********ดูcodeนี้ใหม่*********

In [13]:
def tokenize_and_split_sentences(dataframe, text_column_name):
    # Create a new DataFrame to store the tokenized sentences
    new_dataframe = pd.DataFrame(columns=dataframe.columns)
    rows_to_concat = []
    
    # Iterate over each row in the original DataFrame
    for _, row in dataframe.iterrows():
        text = row[text_column_name]
        text = re.sub(r"\n", '', text)
        text = re.sub(r"\r", '', text)
        sentences = sent_tokenize(text)
        
        # Create a new row for each tokenized sentence
        for sentence in sentences:
            new_row = row.copy()
            new_row[text_column_name] = sentence
            rows_to_concat.append(new_row)
    
    # Convert the list of rows into a DataFrame
    new_dataframe = pd.DataFrame(rows_to_concat)
    new_dataframe.reset_index(drop=True, inplace=True)
    
    return new_dataframe

In [24]:
df_sent = tokenize_and_split_sentences(df, 'textDisplay')
df_sent = df_sent.drop_duplicates()
print("จำนวนข้อมูล",f'{df_sent.shape[0]:,}',"แถว")

จำนวนข้อมูล 4,063 แถว


In [25]:
df_sent["Filter_keyword"] = df_sent["textDisplay"].apply(word_token)
df_sent.head()

,textDisplay,sentiment,Filter_keyword
0,ชอบกล้อง กับ ตัดต่อวีดีโอแค่นั้นแหละถ้าจะซื้อไ...,pos,"[ชอบ, กล้อง, กับ, ตัดต่อ, วีดีโอ, แค่, นั้นแหล..."
1,อยากทราบวิธีรักษาแบต 14 กับความร้อนของเครื่องค...,neg,"[อยากทราบ, วิธี, รักษา, แบต, 14, กับ, ความร้อน..."
2,นี่ซื้อมาใช้ได้ 1 เดือน,NaN,"[นี่, ซื้อ, มา, ใช้ได้, 1, เดือน]"
3,คุณภาพแบตลดไป 3% เหลือ 97%ถ้าหมดไป1ปี,NaN,"[คุณภาพ, แบต, ลด, ไป, 3, เหลือ, 97, ถ้า, หมด, ..."
4,มันจะเหลือถึง 70%ไหมน้อผมเป็นสาวกฝั่งแอนดรอยด์...,NaN,"[มัน, จะ, เหลือ, ถึง, 70, ไหม, น้อผม, เป็น, สา..."


In [26]:
df_sent['Filter_keyword'] = df_sent['Filter_keyword'].apply(keyword_filter)
df_sent = df_sent.dropna()
df_sent.reset_index(drop=True, inplace=True)
df_sent = df_sent.drop(columns="Filter_keyword")
print("จำนวนข้อมูล",f'{df_sent.shape[0]:,}',"แถว")
df_sent.head()

จำนวนข้อมูล 1,196 แถว


,textDisplay,sentiment
0,ชอบกล้อง กับ ตัดต่อวีดีโอแค่นั้นแหละถ้าจะซื้อไ...,pos
1,อยากทราบวิธีรักษาแบต 14 กับความร้อนของเครื่องค...,neg
2,กล้องวูบวาบเป็นเพราะอะไรคะ,neg
3,ผมผู้ถือ 7Plus เมื่อไหร่มันจะเต็มจอสักที,neg
4,สงสัยต้อง 15 เอา แค่ c ก็พอก่อนละม้าง ไอ 18 ค่...,neg


In [27]:
df_questionable_filter = question_filter(df_sent, "textDisplay")
df_questionable_filter.reset_index(drop=True, inplace=True)
print("จำนวนข้อมูล",f'{df_questionable_filter.shape[0]:,}',"แถว")
df_questionable_filter

จำนวนข้อมูล 1,138 แถว


,textDisplay,sentiment
0,ชอบกล้อง กับ ตัดต่อวีดีโอแค่นั้นแหละถ้าจะซื้อไ...,pos
1,ผมผู้ถือ 7Plus เมื่อไหร่มันจะเต็มจอสักที,neg
2,สงสัยต้อง 15 เอา แค่ c ก็พอก่อนละม้าง ไอ 18 ค่...,neg
3,ก็เงินเขาอะเขาก็ผิดหวังปะชิปตัวเดิมจาก13จอ60Hz...,neg
4,หน้าจอควรจะได้,neg
...,...,...
1133,สำหรับออย เอาเครื่องคิดเลขถ่ายก็สวยครับ,pos
1134,"มันคุ้มกว่าแน่นอน เพิ่ม แค่ 3,000 ก็ได้หน้าจอท...",pos
1135,เซนเซอร์กล้องที่ ใหญ่กว่า และ ดีกว่าเยอะ พอควร.,pos
1136,แบตก็อึดกว่า แต่ๆๆๆๆๆ.,pos


In [28]:
df_questionable_filter.to_excel("Corpus_after_remove_question_clean.xlsx")

In [39]:
text = 'หนักว่าบาก เสยเลยครับ 14 plus แบตเยอะกว่า มั้ยครับ'

a = word_tokenize(text)
print(a)

['หนัก', 'ว่า', 'บาก', ' ', 'เสย', 'เลย', 'ครับ', ' ', '14', ' ', 'plus', ' ', 'แบต', 'เยอะ', 'กว่า', ' ', 'มั้ย', 'ครับ']


In [1]:
from langdetect import detect
def is_thai(text):
    try:
        return detect(text) == 'th'
    except:
        return False

In [3]:
test = is_thai("ฉันชอบ iPhone14")
if test == "th":

    print(test)